In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 25.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 6.6 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039180 sha256=ace212252b7c218aab6c1e11a04ca03604f21397d7fa3e7c2d5c68bb0437a16f
  Stored in di

In [ ]:
!pip install bertopic[visualization]

In [ ]:
import pandas as pd
import numpy as np
import json
from bertopic import BERTopic

In [ ]:
# Opening JSON file
f = open('complaints-2021-05-14_08_16_.json')
datafile = json.load(f)
df = pd.json_normalize(datafile)

In [ ]:
df.head()

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,...,Closed with explanation,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,...,Closed with explanation,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A


In [ ]:
#Assign nan in place of blanks in the body column
df[df.loc[:, '_source.complaint_what_happened'] == ''] = np.nan

In [ ]:
# Check if blank values still exist
df[df.loc[:, '_source.complaint_what_happened'] == '']

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company_response,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided


In [ ]:
df.shape

(78313, 22)

In [ ]:
#Remove all rows where body column is nan
df = df[~df['_source.complaint_what_happened'].isnull()]

In [ ]:
df.shape

(21072, 22)

In [ ]:
# Convert body column to string for performing text operations
df['_source.complaint_what_happened'] = df['_source.complaint_what_happened'].astype(str)

In [ ]:
# Write your function here to clean the text and remove all the unnecessary elements.
def clean_text(sent):
    sent = sent.lower() # Text to lowercase
    pattern = '[^\w\s]' # Removing punctuation
    sent = re.sub(pattern, '', sent)
    pattern = '\w*\d\w*' # Removing words with numbers in between
    sent = re.sub(pattern, '', sent)
    return sent

In [ ]:
import re
df_clean = pd.DataFrame(df['_source.complaint_what_happened'].apply(clean_text))
# df_clean.columns = ['complaint_what_happened']

In [ ]:
df_clean.head()

,_source.complaint_what_happened
1,good morning my name is xxxx xxxx and i apprec...
2,i upgraded my xxxx xxxx card in and was told ...
10,chase card was reported on however fraudulent...
11,on while trying to book a xxxx xxxx ticket ...
14,my grand son give me check for i deposit it i...


In [ ]:
# create model
bert_model = BERTopic(verbose=True)
#convert to list
docs = df_clean['_source.complaint_what_happened'].to_list()

In [ ]:
#bert_model.fit_transform(docs)
topics, probabilities = bert_model.fit_transform(docs)

2023-12-12 18:16:04,432 - BERTopic - Embedding - Transforming documents to embeddings.


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/659 [00:00<?, ?it/s]

2023-12-12 19:10:02,449 - BERTopic - Embedding - Completed ✓
2023-12-12 19:10:02,454 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-12 19:11:01,722 - BERTopic - Dimensionality - Completed ✓
2023-12-12 19:11:01,726 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-12 19:11:06,211 - BERTopic - Cluster - Completed ✓
2023-12-12 19:11:06,232 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-12 19:11:17,159 - BERTopic - Representation - Completed ✓


In [ ]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

# Preprocess documents
cleaned_docs = bert_model._preprocess_text(docs)

# Extract vectorizer and tokenizer from BERTopic
vectorizer = bert_model.vectorizer_model
tokenizer = vectorizer.build_tokenizer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names_out()
tokens = [tokenizer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in bert_model.get_topic(topic)]
               for topic in range(len(set(topics))-1)]

In [ ]:
# Evaluate
coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='c_v')
coherence = coherence_model.get_coherence()
coherence

0.5304336847668605

In [ ]:
# Evaluate
u_mass_coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='u_mass')
u_mass_coherence = u_mass_coherence_model.get_coherence()
u_mass_coherence

-2.0991738393963684

In [ ]:
# Evaluate
c_uci_coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='c_uci')
c_uci_coherence = c_uci_coherence_model.get_coherence()
c_uci_coherence

-0.16214295288344877

In [ ]:
# Evaluate
c_npmi_coherence_model = CoherenceModel(topics=topic_words,
                                 texts=tokens,
                                 corpus=corpus,
                                 dictionary=dictionary,
                                 coherence='c_npmi')
c_npmi_coherence = c_npmi_coherence_model.get_coherence()
c_npmi_coherence

0.052055372170235786

In [ ]:
bert_model.get_topic_freq().head(11)

,Topic,Count
0,-1,9239
24,0,1214
7,1,702
14,2,695
9,3,461
53,4,409
8,5,336
42,6,326
52,7,322
1,8,304


In [ ]:
bert_model.get_topic(7)

[('charges', 0.020244036713973217),
 ('charge', 0.013185950293615205),
 ('card', 0.012735738048743625),
 ('fraudulent', 0.012402159623782147),
 ('fraud', 0.010552823126942617),
 ('xxxx', 0.007860165663693287),
 ('that', 0.007591016239874271),
 ('on', 0.007371886854056404),
 ('chase', 0.007042722292726619),
 ('were', 0.006956869461476829)]

In [ ]:
bert_model.visualize_topics()

In [ ]:
bert_model.visualize_barchart()

In [ ]:
bert_model.visualize_heatmap()

In [ ]:
bert_model.save("complaints_bertmodel")

2023-12-12 19:22:43,081 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.
